In [271]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

In [272]:
pred_basel = pd.read_csv("basel.csv",index_col=0)
pred_basel

,datetime,et,p,t,obs
0,1981-01-01,0.000405,0.003536,-0.250816,670.000
1,1981-01-02,0.000472,0.003379,2.172385,647.000
2,1981-01-03,0.000576,0.017352,5.165919,727.000
3,1981-01-04,0.000443,0.022734,1.019687,1363.000
4,1981-01-05,0.000364,0.005195,-2.055391,1202.000
...,...,...,...,...,...
7300,2000-12-27,0.000492,0.001242,2.119268,662.035
7301,2000-12-28,0.000413,0.000561,-0.594725,656.253
7302,2000-12-29,0.000348,0.002027,-3.251007,652.150
7303,2000-12-30,0.000306,0.000741,-5.414832,641.549


In [273]:
q_basel = pd.read_csv("q_basel.csv",index_col=0)
q_basel

,obs,pcr,res
datetime,,,
1981-01-01,670.000,605.06915,64.93085
1981-01-02,647.000,599.35767,47.64233
1981-01-03,727.000,874.38354,-147.38354
1981-01-04,1363.000,998.86804,364.13196
1981-01-05,1202.000,968.06270,233.93730
...,...,...,...
2000-12-27,662.035,1055.62610,-393.59110
2000-12-28,656.253,1027.90730,-371.65430
2000-12-29,652.150,860.76306,-208.61306


In [274]:
# from datetime import datetime
#
# #define function to find the year day
# def date_to_yday(date):
#   datetime_obj = datetime.strptime(date, '%Y-%m-%d')
#   yday = datetime_obj.timetuple().tm_yday
#   return yday
#
# #add day of the year in predictors
# pred_basel['year_day'] = pred_basel['datetime'].apply(date_to_yday)
# pred_basel

In [275]:
# X_train = pred_basel.iloc[]

In [276]:
#define number of observations and the train split proportion
TRAIN_SPLIT = 0.8
nobs = pred_basel.shape[0]
# nobs *TRAIN_SPLIT
#split train and test for predictors vars
X_train = pred_basel.iloc[:int(nobs*TRAIN_SPLIT) , 1:-1].values
X_test = pred_basel.iloc[int(nobs*TRAIN_SPLIT): , 1:-1].values

#split train and test for predicted var
y_train = pred_basel.iloc[:int(nobs*TRAIN_SPLIT) , -1].values
y_test = pred_basel.iloc[int(nobs*TRAIN_SPLIT): , -1].values

In [278]:
y_test

array([1552.   , 1368.   , 1600.   , ...,  652.15 ,  641.549,  599.663])

### normalize the data

In [279]:
from sklearn.preprocessing import MinMaxScaler


### Model

In [280]:
# creating an object of LinearRegression class
lm = LinearRegression()
# fitting the training data
lm.fit(X_train,y_train)

#evaluate the model
lm.score(X_test, y_test)

0.09349916822243631

### evaluation

In [281]:
import hydroeval as he

#compute the streamflow prediction by the coulpled model
y_pred =  lm.predict(X_test)

streamflow_pcr = q_basel.iloc[int(nobs*TRAIN_SPLIT): , -2].values
streamflow_pred = y_pred
streamflow_obs = q_basel.iloc[int(nobs*TRAIN_SPLIT): , 0].values

#evaluate pcr model using nse and kge
nse_pcr = he.evaluator(he.nse, streamflow_pcr, streamflow_obs)
kge_pcr, r, alpha, beta = he.evaluator(he.kge, streamflow_pcr, streamflow_obs)
print("The nse and kge of the PCR model are {:.2} and {:.2}.".format(nse_pcr[0], kge_pcr[0]))

#evaluate the prediction using nse and kge
nse = he.evaluator(he.nse, streamflow_pred, streamflow_obs)

kge, r, alpha, beta = he.evaluator(he.kge, streamflow_pred, streamflow_obs)
print("The nse and kge of the linear model are {:.2} and {:.2}.".format(nse[0], kge[0]))

The nse and kge of the PCR model are 0.19 and 0.62.
The nse and kge of the linear model are 0.093 and 0.15.


In [282]:
streamflow_pcr

array([1285.0109 , 1323.4    , 1311.2427 , ...,  860.76306,  655.83777,
        620.7806 ])